# Gold standard curation: Preprocessing and single-step regression

In this stage of gold standard curation, we will do the data preprocessing, selection, and single-step regression for the 153 traits in our question set. The workflow consists of the following steps:

1. Preprocess all the cohorts related to this trait. Each cohort should be converted to a tabular form and saved to a csv file, with columns being genetic factors, the trait, and age, gender if available;
2. If there exists at least one cohort with age or gender information, conduct regression analysis with genetic features together with age or gender as the regressors.


# 1. Basic setup

In [1]:
import os
import sys

sys.path.append('..')
# Set your preferred name
USER = "Arthur"
# Set the data and output directories
DATA_ROOT = 'C:/Users/arthu/Downloads'
OUTPUT_ROOT = 'C:/Users/arthu/OneDrive/Documents/GitHub/output1'
TRAIT = 'Obesity'

OUTPUT_DIR = os.path.join(OUTPUT_ROOT, USER, '-'.join(TRAIT.split()))
JSON_PATH = os.path.join(OUTPUT_DIR, "cohort_info.json")
if not os.path.exists(OUTPUT_DIR):
    os.makedirs(OUTPUT_DIR, exist_ok=True)

# Gene symbol normalization may take 1-2 minutes. You may set it to False for debugging.
NORMALIZE_GENE = True

In [25]:
# This cell is only for use on Google Colab. Skip it if you run your code in other environments

"""import os
from google.colab import drive

drive.mount('/content/drive', force_remount=True)
proj_dir = '/content/drive/MyDrive/AI4Science_Public'
os.chdir(proj_dir)"""

"import os\nfrom google.colab import drive\n\ndrive.mount('/content/drive', force_remount=True)\nproj_dir = '/content/drive/MyDrive/AI4Science_Public'\nos.chdir(proj_dir)"

# 2. Data preprocessing and selection

## 2.1. The TCGA Xena dataset

In TCGA Xena, there is either zero or one cohort related to the trait. We search the names of subdirectories to see if any matches the trait. If a match is found, we directly obtain the file paths.

In [2]:
import os
from utils import *

dataset = 'TCGA'
dataset_dir = os.path.join(DATA_ROOT, dataset)
os.listdir(dataset_dir)[:]

utils.py has been loaded


['TCGA_Colon_and_Rectal_Cancer_(COADREAD)']

If no match is found, jump directly to GEO in Part 2.2

In [27]:
trait_subdir = "TCGA_Lung_Cancer_(LUNG)"
cohort = 'Xena'
# All the cancer traits in Xena are binary
trait_type = 'binary'
# Once a relevant cohort is found in Xena, we can generally assume the gene and clinical data are available
is_available = True

clinical_data_file = os.path.join(dataset_dir, trait_subdir, 'TCGA.LUNG.sampleMap_LUNG_clinicalMatrix')
genetic_data_file = os.path.join(dataset_dir, trait_subdir, 'TCGA.LUNG.sampleMap_HiSeqV2_PANCAN.gz')

In [28]:
import pandas as pd

clinical_data = pd.read_csv(clinical_data_file, sep='\t', index_col=0)
genetic_data = pd.read_csv(genetic_data_file, compression='gzip', sep='\t', index_col=0)
age_col = gender_col = None

ParserError: Error tokenizing data. C error: Calling read(nbytes) on source failed. Try engine='python'.

In [ ]:
_, clinical_data_cols = check_rows_and_columns(clinical_data)

In [ ]:
clinical_data_cols[:10]

['ABSOLUTE_Ploidy',
 'ABSOLUTE_Purity',
 'AKT1',
 'ALK_translocation',
 'BRAF',
 'CBL',
 'CTNNB1',
 'Canonical_mut_in_KRAS_EGFR_ALK',
 'Cnncl_mt_n_KRAS_EGFR_ALK_RET_ROS1_BRAF_ERBB2_HRAS_NRAS_AKT1_MAP2',
 'EGFR']

Read all the column names in the clinical dataset, to find the columns that record information about age or gender.
Reference prompt:

In [ ]:
f'''
Below is a list of column names from a biomedical dataset. Please examine it and identify the columns that are likely to contain information about patients' age. Additionally, please do the same for columns that may hold data on patients' gender. Please provide your answer by strictly following this format, without redundant words:
candidate_age_cols = [col_name1, col_name2, ...]
candidate_gender_cols = [col_name1, col_name2, ...]
If no columns match a criterion, please provide an empty list.

Column names:
{clinical_data_cols}
'''

"\nBelow is a list of column names from a biomedical dataset. Please examine it and identify the columns that are likely to contain information about patients' age. Additionally, please do the same for columns that may hold data on patients' gender. Please provide your answer by strictly following this format, without redundant words:\ncandidate_age_cols = [col_name1, col_name2, ...]\ncandidate_gender_cols = [col_name1, col_name2, ...]\nIf no columns match a criterion, please provide an empty list.\n\nColumn names:\n['ABSOLUTE_Ploidy', 'ABSOLUTE_Purity', 'AKT1', 'ALK_translocation', 'BRAF', 'CBL', 'CTNNB1', 'Canonical_mut_in_KRAS_EGFR_ALK', 'Cnncl_mt_n_KRAS_EGFR_ALK_RET_ROS1_BRAF_ERBB2_HRAS_NRAS_AKT1_MAP2', 'EGFR', 'ERBB2', 'ERBB4', 'Estimated_allele_fraction_of_a_clonal_varnt_prsnt_t_1_cpy_pr_cll', 'Expression_Subtype', 'HRAS', 'KRAS', 'MAP2K1', 'MET', 'NRAS', 'PIK3CA', 'PTPN11', 'Pathology', 'Pathology_Updated', 'RET_translocation', 'ROS1_translocation', 'STK11', 'WGS_as_of_20120731_

In [ ]:
candidate_age_cols = ['age_at_initial_pathologic_diagnosis', 'days_to_birth', 'days_to_collection', 'days_to_death', 'days_to_initial_pathologic_diagnosis', 'days_to_last_followup', 'days_to_new_tumor_event_after_initial_treatment', 'stopped_smoking_year', 'year_of_initial_pathologic_diagnosis', 'year_of_tobacco_smoking_onset']

candidate_gender_cols = ['gender']

Choose a single column from the candidate columns that record age and gender information respectively.
If no column meets the requirement, keep 'age_col' or 'gender_col' to None

In [ ]:
preview_df(clinical_data[candidate_age_cols])

{'age_at_initial_pathologic_diagnosis': [70.0, 67.0, 79.0, 68.0, 66.0],
 'days_to_birth': [-25752.0, -24532.0, -29068.0, -24868.0, -24411.0],
 'days_to_collection': [nan, nan, nan, nan, nan],
 'days_to_death': [nan, nan, '121', nan, nan],
 'days_to_initial_pathologic_diagnosis': [0.0, 0.0, 0.0, 0.0, 0.0],
 'days_to_last_followup': ['0', '1523', nan, '607', '426'],
 'days_to_new_tumor_event_after_initial_treatment': [nan,
  nan,
  nan,
  334.0,
  183.0],
 'stopped_smoking_year': [nan, nan, nan, nan, 1987.0],
 'year_of_initial_pathologic_diagnosis': [2009.0,
  2007.0,
  2007.0,
  2009.0,
  2009.0],
 'year_of_tobacco_smoking_onset': [nan, nan, nan, nan, 1963.0]}

In [ ]:
age_col = 'age_at_initial_pathologic_diagnosis'

In [ ]:
preview_df(clinical_data[candidate_gender_cols])

{'gender': ['MALE', 'MALE', 'FEMALE', 'MALE', 'MALE']}

In [ ]:
gender_col = 'gender'

In [ ]:
selected_clinical_data = xena_select_clinical_features(clinical_data, TRAIT, age_col, gender_col)

In [ ]:
if NORMALIZE_GENE:
    genetic_data = normalize_gene_symbols_in_index(genetic_data)

In [ ]:
merged_data = selected_clinical_data.join(genetic_data.T).dropna()
merged_data.head()

,Lung-Cancer,Age,Gender,ARHGEF10L,HIF3A,RNF17,RNF10,RNF11,RNF13,GTF2IP1,...,TULP2,NPY5R,GNGT2,GNGT1,TULP3,PTRF,BCL6B,GSTK1,SELP,SELS
sampleID,,,,,,,,,,,,,,,,,,,,,
TCGA-05-4244-01,1,70.0,1.0,-0.056592,2.128674,-0.531035,-0.185972,-0.190878,-0.11891,-0.659794,...,0.015022,-1.155117,0.528367,2.75061,0.428423,-0.639586,-0.195527,-0.324695,0.193167,-0.572812
TCGA-05-4249-01,1,67.0,1.0,-0.569692,-0.318126,-0.531035,0.237428,0.011822,-0.00611,1.018406,...,-0.031778,-0.393117,1.632467,2.33551,-0.145577,-0.997486,-0.276027,-1.219395,1.850967,-0.471212
TCGA-05-4250-01,1,79.0,0.0,-0.899992,-0.918126,-0.531035,-0.621472,-0.454178,0.58719,-0.198894,...,-0.748878,-1.173917,1.966367,-1.28139,-0.374877,0.959514,0.714373,0.112305,-0.532833,-0.003812
TCGA-05-4382-01,1,68.0,1.0,0.667508,3.895974,-0.221535,-0.367272,0.131822,0.97629,-0.494994,...,1.013722,-1.587117,0.359267,2.59051,0.806623,0.812114,0.367373,-0.494595,-1.062233,0.108588
TCGA-05-4384-01,1,66.0,1.0,-0.316892,2.380374,0.156865,0.232828,0.054822,0.27059,0.128606,...,-0.060978,-0.435417,1.111467,-1.28139,-0.128477,0.110214,1.469373,0.194605,1.404667,-0.290712


In [ ]:
print(f"The merged dataset contains {len(merged_data)} samples.")

The merged dataset contains 1100 samples.


In [ ]:
is_trait_biased, merge_data = judge_and_remove_biased_features(merged_data, TRAIT, trait_type=trait_type)
is_trait_biased

For the feature 'Lung-Cancer', the least common label is '0' with 110 occurrences. This represents 10.00% of the dataset.
The distribution of the feature 'Lung-Cancer' in this dataset is fine.

Quartiles for 'Age':
  25%: 60.0
  50% (Median): 67.0
  75%: 73.0
Min: 38.0
Max: 90.0
The distribution of the feature 'Age' in this dataset is fine.

For the feature 'Gender', the least common label is '0.0' with 443 occurrences. This represents 40.27% of the dataset.
The distribution of the feature 'Gender' in this dataset is fine.



False

In [ ]:
merged_data.head()
if not is_trait_biased:
    merge_data.to_csv(os.path.join(OUTPUT_DIR, cohort+'.csv'), index=False)

In [ ]:
save_cohort_info(cohort, JSON_PATH, is_available, is_trait_biased, merged_data)

A new JSON file was created at: ../../output1/Yuxuan/Lung-Cancer/cohort_info.json


## 2.2. The GEO dataset

In GEO, there may be one or multiple cohorts for a trait. Each cohort is identified by an accession number. We iterate over all accession numbers in the corresponding subdirectory, preprocess the cohort data, and save them to csv files.

In [3]:
dataset = 'GEO'
trait_subdir = "Obesity"

trait_path = os.path.join(DATA_ROOT, dataset, trait_subdir)
os.listdir(trait_path)

['GSE176025', 'GSE181339', 'GSE242923']

Repeat the below steps for all the accession numbers

In [4]:
cohort = accession_num = "GSE242923"
cohort_dir = os.path.join(trait_path, accession_num)
soft_file, matrix_file = geo_get_relevant_filepaths(cohort_dir)
soft_file, matrix_file

('C:/Users/arthu/Downloads\\GEO\\Obesity\\GSE242923\\GSE242923_family.soft.gz',
 'C:/Users/arthu/Downloads\\GEO\\Obesity\\GSE242923\\GSE242923_series_matrix.txt.gz')

### Inital filtering and clinical data preprocessing

In [5]:
background_prefixes = ['!Series_title', '!Series_summary', '!Series_overall_design']
clinical_prefixes = ['!Sample_geo_accession', '!Sample_characteristics_ch1']

background_info, clinical_data = get_background_and_clinical_data(matrix_file, background_prefixes, clinical_prefixes)
print(background_info)

!Series_title	"miRNA expression profile from Adipocyte-derived Small Extracellular Vesicles from the Serum and cerebrospinal fluid of Alzheimer Disease Patients and Controls"
!Series_summary	"Midlife obesity increases the risk of developing AD. Adipocyte-derived small extracellular vesicles (ad-sEVs) have been implicated as a mecha-nism in several obesity-related diseases. We hypothesized that ad-sEVs from patients with AD would contain miRNAs predicted to downregulate pathways involved in synaptic plasticity and memory formation. We isolated ad-sEVs from the serum and cerebrospinal fluid (CSF) of patients with AD and controls and compared miRNA expression profiles"
!Series_overall_design	"Ad-sEV miRNA was isolated from the serum (n=12 controls, n=17 AD) and cerebrospinal fluid (n=10 controls, n=14 AD). Total RNA was extracted using mirVana microRNA Isolation kits (Life Technologies). Total RNA was amplified with the Complete Seramir Exosome RNA Amplification Kit (System Biosciences, M

In [6]:
clinical_data.head()

,!Sample_geo_accession,GSM7775050,GSM7775051,GSM7775052,GSM7775053,GSM7775054,GSM7775055,GSM7775056,GSM7775057,GSM7775058,...,GSM7775093,GSM7775094,GSM7775095,GSM7775096,GSM7775097,GSM7775098,GSM7775099,GSM7775100,GSM7775101,GSM7775102
0,!Sample_characteristics_ch1,tissue: Adipocyte-dervied Extracellular Vesicl...,tissue: Adipocyte-dervied Extracellular Vesicl...,tissue: Adipocyte-dervied Extracellular Vesicl...,tissue: Adipocyte-dervied Extracellular Vesicl...,tissue: Adipocyte-dervied Extracellular Vesicl...,tissue: Adipocyte-dervied Extracellular Vesicl...,tissue: Adipocyte-dervied Extracellular Vesicl...,tissue: Adipocyte-dervied Extracellular Vesicl...,tissue: Adipocyte-dervied Extracellular Vesicl...,...,tissue: Adipocyte-dervied Extracellular Vesicl...,tissue: Adipocyte-dervied Extracellular Vesicl...,tissue: Adipocyte-dervied Extracellular Vesicl...,tissue: Adipocyte-dervied Extracellular Vesicl...,tissue: Adipocyte-dervied Extracellular Vesicl...,tissue: Adipocyte-dervied Extracellular Vesicl...,tissue: Adipocyte-dervied Extracellular Vesicl...,tissue: Adipocyte-dervied Extracellular Vesicl...,tissue: Adipocyte-dervied Extracellular Vesicl...,tissue: Adipocyte-dervied Extracellular Vesicl...
1,!Sample_characteristics_ch1,disease state: Alzheimer Disease,disease state: Alzheimer Disease,disease state: Alzheimer Disease,disease state: Control,disease state: Control,disease state: Control,disease state: Control,disease state: Control,disease state: Control,...,disease state: Alzheimer Disease,disease state: Alzheimer Disease,disease state: Alzheimer Disease,disease state: Alzheimer Disease,disease state: Alzheimer Disease,disease state: Alzheimer Disease,disease state: Alzheimer Disease,disease state: Alzheimer Disease,disease state: Alzheimer Disease,disease state: Alzheimer Disease


In [7]:
clinical_data_unique = get_unique_values_by_row(clinical_data)
clinical_data_unique

{0: ['tissue: Adipocyte-dervied Extracellular Vesicles_CSF',
  'tissue: Adipocyte-dervied Extracellular Vesicles_Serum'],
 1: ['disease state: Alzheimer Disease', 'disease state: Control']}

Analyze the metadata to determine data relevance and find ways to extract the clinical data.
Reference prompt:

In [8]:
f'''As a biomedical research team, we are selecting datasets to study the association between the human trait \'{TRAIT}\' and genetic factors, optionally considering the influence of age and gender. After searching the GEO database and parsing the matrix file of a series, we obtained background information and sample characteristics data. We will provide textual information about the dataset background, and a Python dictionary storing a list of unique values for each field of the sample characteristics data. Please carefully review the provided information and answer the following questions about this dataset:
1. Does this dataset contain gene expression data? (Note: Pure miRNA data is not suitable.)
2. For each of the traits \'{TRAIT}\', 'age', and 'gender', please address these points:
   (1) Is there human data available for this trait?
   (2) If so, identify the key in the sample characteristics dictionary where unique values of this trait is recorded. The key is an integer. The trait information might be explicitly recorded, or can be inferred from the field with some biomedical knowledge or understanding about the data collection process.
   (3) Choose an appropriate data type (either 'continuous' or 'binary') for each trait. Write a Python function to convert any given value of the trait to this data type. The function should handle inference about the trait value and convert unknown values to None.
   Name the functions 'convert_trait', 'convert_age', and 'convert_gender', respectively.

Background information about the dataset:
{background_info}

Sample characteristics dictionary (from "!Sample_characteristics_ch1", converted to a Python dictionary that stores the unique values for each field):
{clinical_data_unique}
'''

'As a biomedical research team, we are selecting datasets to study the association between the human trait \'Obesity\' and genetic factors, optionally considering the influence of age and gender. After searching the GEO database and parsing the matrix file of a series, we obtained background information and sample characteristics data. We will provide textual information about the dataset background, and a Python dictionary storing a list of unique values for each field of the sample characteristics data. Please carefully review the provided information and answer the following questions about this dataset:\n1. Does this dataset contain gene expression data? (Note: Pure miRNA data is not suitable.)\n2. For each of the traits \'Obesity\', \'age\', and \'gender\', please address these points:\n   (1) Is there human data available for this trait?\n   (2) If so, identify the key in the sample characteristics dictionary where unique values of this trait is recorded. The key is an integer. T

Understand and verify the answer from GPT, to assign values to the below variables. Assign None to the 'row_id' variables if relevant data row was not found.
Later we need to let GPT format its answer to automatically do these. But given the complexity of this step, let's grow some insight from the free-text answers for now.

In [9]:
is_gene_availabe = False
trait_row = 2
age_row = None
gender_row = None

trait_type = 'binary'

In [10]:
is_available = is_gene_availabe and (trait_row is not None)
if not is_available:
    save_cohort_info(cohort, JSON_PATH, is_available)
    print("This cohort is not usable. Please skip the following steps and jump to the next accession number.")

This cohort is not usable. Please skip the following steps and jump to the next accession number.


In [47]:
def convert_trait(tissue_type):
    """
    Convert tissue type to epilepsy presence (binary).
    Assuming epilepsy presence for 'Hippocampus' tissue.
    """
    if tissue_type == 'group: NW':
        return 0  # Epilepsy present
    else:
        return 1  # Epilepsy not present

def convert_age(age_string):
    """
    Convert age string to a numerical value.
    """
    if age_string.startswith('age: '):
        try:
            return int(age_string.split('age: ')[1].rstrip(''))  # Extracting numeric part
        except ValueError:
            return None  # In case of parsing error
    else:
        return None  # If age string does not follow expected format

def convert_gender(gender_string):
    """
    Convert gender string to a binary representation.
    Female -> 0, Male -> 1, Unknown -> None
    """
    if gender_string == 'gender: Woman':
        return 0
    elif gender_string == 'gender: Man':
        return 1
    else:
        return None  # In case of unknown gender

# Example usage
example_tissue = 'tissue: Hippocampus'
example_age = 'age: 23y'
example_gender = 'gender: Male'

converted_trait = convert_trait(example_tissue)
converted_age = convert_age(example_age)
converted_gender = convert_gender(example_gender)

converted_trait, converted_age, converted_gender




(1, None, None)

In [48]:
selected_clinical_data = geo_select_clinical_features(clinical_data, TRAIT, trait_row, convert_trait, age_row=age_row,
                                                      convert_age=convert_age, gender_row=gender_row,
                                                      convert_gender=convert_gender)
selected_clinical_data.head()

,GSM5494930,GSM5494931,GSM5494932,GSM5494933,GSM5494934,GSM5494935,GSM5494936,GSM5494937,GSM5494938,GSM5494939,...,GSM5494998,GSM5494999,GSM5495000,GSM5495001,GSM5495002,GSM5495003,GSM5495004,GSM5495005,GSM5495006,GSM5495007
Obesity,0,1,1,1,0,1,1,1,0,1,...,1,1,1,0,1,0,1,1,1,0
Age,21,23,10,17,11,1,18,10,12,23,...,2,2,30,17,30,19,30,2,4,19
Gender,1,1,0,0,0,1,0,0,0,1,...,0,0,1,1,1,1,1,0,1,1


### Genetic data preprocessing and final filtering

In [49]:
genetic_data = get_genetic_data(matrix_file)
genetic_data.head()

,GSM5494930,GSM5494931,GSM5494932,GSM5494933,GSM5494934,GSM5494935,GSM5494936,GSM5494937,GSM5494938,GSM5494939,...,GSM5494998,GSM5494999,GSM5495000,GSM5495001,GSM5495002,GSM5495003,GSM5495004,GSM5495005,GSM5495006,GSM5495007
ID,,,,,,,,,,,,,,,,,,,,,
7,5.505549,5.513184,4.986265,5.346870,5.531071,4.848168,5.063753,5.133687,6.971369,6.581063,...,5.930267,5.864811,6.180203,6.302292,6.279264,6.346574,7.841532,6.446191,6.009474,5.820668
8,14.922929,14.996884,15.558723,15.464692,14.914828,14.921011,14.656432,15.027612,15.462935,14.993909,...,14.720862,14.991377,15.062609,14.941717,15.565374,15.562799,15.232485,14.845854,14.843489,15.120225
15,10.185238,9.826678,9.003106,10.440502,8.448914,9.775237,9.587976,8.860986,8.913982,9.646833,...,9.531411,9.137210,10.327923,9.672288,9.841389,9.210932,9.746890,8.812242,8.952330,8.907866
18,10.064262,10.235713,9.925906,9.915432,9.793022,9.425392,9.561722,9.583471,10.332012,9.625713,...,9.351078,9.505807,9.616248,9.689660,9.452713,9.733175,9.733940,9.405109,9.631529,9.408484
20,6.339479,6.272158,6.426318,6.371126,6.117284,5.886479,5.469945,5.672563,7.095820,6.951400,...,6.341808,6.790785,6.745134,7.003243,6.966419,6.912556,6.632631,6.584196,6.423572,6.826112


In [50]:
gene_row_ids = genetic_data.index[:20].tolist()
gene_row_ids

['7',
 '8',
 '15',
 '18',
 '20',
 '21',
 '24',
 '25',
 '29',
 '32',
 '39',
 '41',
 '42',
 '44',
 '45',
 '46',
 '48',
 '51',
 '53',
 '55']

Check if the gene dataset requires mapping to get the gene symbols corresponding to each data row.

Reference prompt:

In [51]:
f'''
Below are the row headers of a gene expression dataset in GEO. Based on your biomedical knowledge, are they human gene symbols, or are they some other identifiers that need to be mapped to gene symbols? Your answer should be concluded by starting a new line and strictly following this format:
requires_gene_mapping = (True or False)

Row headers:
{gene_row_ids}
'''

"\nBelow are the row headers of a gene expression dataset in GEO. Based on your biomedical knowledge, are they human gene symbols, or are they some other identifiers that need to be mapped to gene symbols? Your answer should be concluded by starting a new line and strictly following this format:\nrequires_gene_mapping = (True or False)\n\nRow headers:\n['7', '8', '15', '18', '20', '21', '24', '25', '29', '32', '39', '41', '42', '44', '45', '46', '48', '51', '53', '55']\n"

If not required, jump directly to the gene normalization step

In [52]:
requires_gene_mapping = True

In [53]:
if requires_gene_mapping:
    gene_annotation = get_gene_annotation(soft_file)
    gene_annotation_summary = preview_df(gene_annotation)
    print(gene_annotation_summary)

{'ID': ['1', '2', '3', '4', '5'], 'COL': ['192', '192', '192', '192', '192'], 'ROW': [328.0, 326.0, 324.0, 322.0, 320.0], 'NAME': ['GE_BrightCorner', 'DarkCorner', 'DarkCorner', 'A_21_P0014386', 'A_33_P3396872'], 'SPOT_ID': ['GE_BrightCorner', 'DarkCorner', 'DarkCorner', 'A_21_P0014386', 'A_33_P3396872'], 'CONTROL_TYPE': ['pos', 'pos', 'pos', 'FALSE', 'FALSE'], 'REFSEQ': [nan, nan, nan, nan, 'NM_001105533'], 'GB_ACC': [nan, nan, nan, nan, 'NM_001105533'], 'LOCUSLINK_ID': [nan, nan, nan, nan, 79974.0], 'GENE_SYMBOL': [nan, nan, nan, nan, 'CPED1'], 'GENE_NAME': [nan, nan, nan, nan, 'cadherin-like and PC-esterase domain containing 1'], 'UNIGENE_ID': [nan, nan, nan, nan, 'Hs.189652'], 'ENSEMBL_ID': [nan, nan, nan, nan, nan], 'TIGR_ID': [nan, nan, nan, nan, nan], 'ACCESSION_STRING': [nan, nan, nan, nan, 'ref|NM_001105533|gb|AK025639|gb|BC030538|tc|THC2601673'], 'CHROMOSOMAL_LOCATION': [nan, nan, nan, 'unmapped', 'chr7:120901888-120901947'], 'CYTOBAND': [nan, nan, nan, nan, 'hs|7q31.31'], 'D

Observe the first few cells in the ID column of the gene annotation dataframe, to find the names of columns that store the gene probe IDs and gene symbols respectively.
Reference prompt:

In [54]:
if requires_gene_mapping:
    print(f'''
    As a biomedical research team, we extracted the gene annotation data from a series in the GEO database, and saved it to a Python dictionary. Please read the dictionary, and decide which key stores the ID of the probe, and which key stores the gene symbols. Please strict follow this format in your answer:
    probe_name_key = 'key_name1'
    gene_name_key = 'key_name2'

    Gene annotation dictionary:
    {gene_annotation_summary}
    ''')


    As a biomedical research team, we extracted the gene annotation data from a series in the GEO database, and saved it to a Python dictionary. Please read the dictionary, and decide which key stores the ID of the probe, and which key stores the gene symbols. Please strict follow this format in your answer:
    probe_name_key = 'key_name1'
    gene_name_key = 'key_name2'

    Gene annotation dictionary:
    {'ID': ['1', '2', '3', '4', '5'], 'COL': ['192', '192', '192', '192', '192'], 'ROW': [328.0, 326.0, 324.0, 322.0, 320.0], 'NAME': ['GE_BrightCorner', 'DarkCorner', 'DarkCorner', 'A_21_P0014386', 'A_33_P3396872'], 'SPOT_ID': ['GE_BrightCorner', 'DarkCorner', 'DarkCorner', 'A_21_P0014386', 'A_33_P3396872'], 'CONTROL_TYPE': ['pos', 'pos', 'pos', 'FALSE', 'FALSE'], 'REFSEQ': [nan, nan, nan, nan, 'NM_001105533'], 'GB_ACC': [nan, nan, nan, nan, 'NM_001105533'], 'LOCUSLINK_ID': [nan, nan, nan, nan, 79974.0], 'GENE_SYMBOL': [nan, nan, nan, nan, 'CPED1'], 'GENE_NAME': [nan, nan, nan, nan, 

In [55]:
if requires_gene_mapping:
    probe_id_key = 'ID'
    gene_symb_key = 'GENE_SYMBOL'
    gene_mapping = get_gene_mapping(gene_annotation, probe_id_key, gene_symb_key)
    genetic_data = apply_gene_mapping(genetic_data, gene_mapping)

In [56]:
if NORMALIZE_GENE:
    genetic_data = normalize_gene_symbols_in_index(genetic_data)

In [57]:
merged_data = geo_merge_clinical_genetic_data(selected_clinical_data, genetic_data)
# The preprocessing runs through, which means is_available should be True
is_available = True

In [58]:
print(f"The merged dataset contains {len(merged_data)} samples.")

The merged dataset contains 78 samples.


In [59]:
is_trait_biased, merged_data = judge_and_remove_biased_features(merged_data, TRAIT, trait_type=trait_type)
is_trait_biased

For the feature 'Obesity', the least common label is '0.0' with 24 occurrences. This represents 30.77% of the dataset.
The distribution of the feature 'Obesity' in this dataset is fine.

Quartiles for 'Age':
  25%: 8.0
  50% (Median): 11.0
  75%: 18.75
Min: 1.0
Max: 30.0
The distribution of the feature 'Age' in this dataset is fine.

For the feature 'Gender', the least common label is '1.0' with 39 occurrences. This represents 50.00% of the dataset.
The distribution of the feature 'Gender' in this dataset is fine.



False

In [60]:
save_cohort_info(cohort, JSON_PATH, is_available, is_trait_biased, merged_data, note='')

In [61]:
merged_data.head()
if not is_trait_biased:
    merged_data.to_csv(os.path.join(OUTPUT_DIR, cohort+'.csv'), index=False)

### 3. Do regression & Cross Validation

In [27]:
from utils import *

In [28]:
best_cohort, ranked_df = filter_and_rank_cohorts(JSON_PATH, 'has_age')
best_cohort

'GSE181339'

In [29]:
ranked_df.head()

,cohort_id,is_usable,is_available,is_biased,has_age,has_gender,sample_size,note
0,GSE181339,True,True,False,True,True,78,


In [30]:
ck=100
listq=[]
for df in pd.read_csv(os.path.join(OUTPUT_DIR, best_cohort+'.csv'),chunksize=ck):
    listq.append(df)
merged_data = pd.concat(listq)
#merged_data = pd.read_csv(os.path.join(OUTPUT_DIR, best_cohort+'.csv'))

In [31]:
merged_data.head()

,Obesity,Age,Gender,A1BG,A1BG-AS1,A2M-AS1,A4GALT,AAAS,AACS,AADACL3,...,ZW10,ZWILCH,ZWINT,ZXDA,ZXDB,ZXDC,ZYG11B,ZYX,ZZEF1,ZZZ3
0,0.0,21.0,1.0,9.356389,6.588705,10.086590,6.087023,8.855058,6.453003,6.644548,...,7.339372,6.278003,5.738853,5.108455,9.036174,7.664560,6.691724,10.559644,11.370012,8.635409
1,1.0,23.0,1.0,9.580217,6.861172,8.589913,5.958440,8.172307,6.070650,6.281522,...,7.143025,6.687131,6.101444,5.254791,9.805082,7.791538,6.815505,10.200364,11.444456,8.916923
2,1.0,10.0,0.0,9.920784,7.055549,9.467662,6.690681,8.768802,6.703954,7.138168,...,6.888677,6.648450,6.406971,5.426840,8.937783,7.575047,6.709371,11.124261,11.062703,8.439625
3,1.0,17.0,0.0,9.504974,6.792186,7.930585,5.814862,8.708854,6.803403,5.934699,...,7.263604,6.583358,6.144584,5.039489,9.203809,7.625925,6.840306,11.300428,10.987588,8.646433
4,0.0,11.0,0.0,9.533504,7.192053,9.596064,5.822462,8.534389,6.751145,6.999438,...,6.861367,6.475069,6.905870,5.460082,9.331111,8.048430,6.860686,10.561634,10.786569,8.603074


In [32]:
# If both age and gender features are available, select 'age' as the condition.
condition = 'Age'
# Remove the other condition to prevent interference.
merged_data = merged_data.drop(columns=['Gender'], errors='ignore').astype('float')

In [33]:
X = merged_data.drop(columns=[TRAIT, condition]).values
Y = merged_data[TRAIT].values
Z = merged_data[condition].values

Select the appropriate regression model depending on whether the dataset shows batch effect.

In [34]:
has_batch_effect = detect_batch_effect(X)
has_batch_effect

True

In [35]:

if has_batch_effect:
    model_constructor1 = VariableSelection
    model_params1 = {'modified': True, 'lamda': 3e-4}
    model_constructor2 = VariableSelection
    model_params2 = {'modified': False}
else:
    model_constructor1 = Lasso
    model_params1 = {'alpha': 1.0, 'random_state': 42}
    model_constructor2 = VariableSelection
    model_params2 = {'modified': False}

In [36]:
trait_type = 'binary'  # Remember to set this properly, either 'binary' or 'continuous'
cv_mean1, cv_std1 = cross_validation(X, Y, Z, model_constructor1, model_params1, target_type=trait_type)

alpha for Lasso: 0.0003
alpha for Lasso: 0.0003
alpha for Lasso: 0.0003
alpha for Lasso: 0.0003
alpha for Lasso: 0.0003
The cross-validation accuracy is 49.33% ± 16.65%


In [37]:
cv_mean2, cv_std2 = cross_validation(X, Y, Z, model_constructor2, model_params2, target_type=trait_type)

The cross-validation accuracy is 85.33% ± 8.84%


In [38]:
normalized_X, _ = normalize_data(X)
normalized_Z, _ = normalize_data(Z)

# Train regression model on the whole dataset to identify significant genes
model1 = ResidualizationRegressor(model_constructor1, model_params1)
model1.fit(normalized_X, Y, normalized_Z)

model2 = ResidualizationRegressor(model_constructor2, model_params2)
model2.fit(normalized_X, Y, normalized_Z)

alpha for Lasso: 0.0003


### 5. Discussion and report

In [39]:

feature_cols = merged_data.columns.tolist()
feature_cols.remove(TRAIT)

threshold = 0.05
interpret_result(model1, feature_cols, TRAIT, condition, threshold=threshold, save_output=True,
                 output_dir=OUTPUT_DIR, model_id=1)

Effect of the condition on the target variable:
Variable: Age
Coefficient: 0.0224
Found 62 genes with non-zero coefficients associated with the trait 'Obesity' conditional on the factor 'Age'. These genes are identified as significant based on the regression model.
    Variable  Coefficient  Absolute Coefficient
     SLC2A14     8.222422              8.222422
   SNORD3B-1    -6.168790              6.168790
       SAP18    -4.886103              4.886103
     ANAPC13     4.228388              4.228388
    EHD4-AS1     3.891012              3.891012
    CATSPERG     3.782441              3.782441
       CNTD1     3.762989              3.762989
       RAPH1     3.599806              3.599806
    NAPA-AS1     3.529587              3.529587
      MAGEE1     3.390650              3.390650
       TDRG1     3.145982              3.145982
        LY6E     3.019098              3.019098
       LIN54    -3.019071              3.019071
      ZNF561    -2.928046              2.928046
       PEX12  

/home/yxcheng/AI4Science_gold_standard/stage1/Haoyang/../utils.py:469: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  gene_regression_df['Absolute Coefficient'] = gene_regression_df['Coefficient'].abs()


In [40]:
interpret_result(model2, feature_cols, TRAIT, condition, threshold=threshold, save_output=True,
                 output_dir=OUTPUT_DIR, model_id=2)

Effect of the condition on the target variable:
Variable: Age
Coefficient: 0.0224
Found 5 significant genes associated with the trait 'Obesity' conditional on the factor 'Age', with corrected p-value < 0.05:
Variable  Coefficient      p_value  corrected_p_value
  PRSS57     0.217830 2.472877e-07           0.002916
    AMFR     0.221464 6.927355e-06           0.032187
   ELANE     0.221936 1.111987e-05           0.032187
    MLH3    -0.225847 8.796122e-06           0.032187
  ZNF438    -0.195550 1.365030e-05           0.032187


/home/yxcheng/AI4Science_gold_standard/stage1/Haoyang/../utils.py:478: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  gene_regression_df.loc[:, 'corrected_p_value'] = corrected_p_values
